In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import json
import matplotlib.pyplot as plt
import shutil

In [8]:
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
%cd /content/drive/My\ Drive/mvtec_screws/
!pwd

/content/drive/My Drive/mvtec_screws
/content/drive/My Drive/mvtec_screws


In [10]:
with open("./mvtec_screws.json", "r") as f:
    annots = json.load(f)

annotations = annots["annotations"]
categories = annots["categories"]
images = annots["images"]
print("Objects "+str(len(annotations)))
print("Categories "+str(len(categories)))
print("Images "+str(len(images)))

Objects 4427
Categories 13
Images 384


In [11]:
images_dir = './images'
tfrecords_dir = "tfrecords"

In [12]:
file_count = sum([len(files) for r, d, files in os.walk(images_dir)])
num_samples = len(images)
num_tfrecords = file_count // num_samples
print(num_tfrecords)

1


In [13]:
def _image_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[tf.io.encode_png(value).numpy()])
    )

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def _int64_list(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _float_list(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def create_example(image, path, example):
    feature = {
        "image": _image_feature(image),
        "area": _float_list(example["area"]),
        
        #Originally annotated as bounding boxes. Each box contains 5 parameters (row, col, width, height, phi)
        "row": _float_list(example["row"]),
        "col": _float_list(example["col"]),
        "width": _float_list(example["width"]),
        "height": _float_list(example["height"]),
        "phi": _float_list(example["phi"]),

        "category_id": _int64_list(example["category_id"]),
        "id": _int64_list(example["id"]),
        "image_id": _int64_feature(example["image_id"]),
        "is_crowd": _int64_list(example["is_crowd"]),
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))


def parse_tfrecord_fn(example):
    feature_description = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "area": tf.io.FixedLenFeature([], tf.float32),

        #Originally annotated as bounding boxes. Each box contains 5 parameters (row, col, width, height, phi)
        "row": tf.io.FixedLenFeature([], tf.float32),
        "col": tf.io.FixedLenFeature([], tf.float32),
        "width": tf.io.FixedLenFeature([], tf.float32),
        "height": tf.io.FixedLenFeature([], tf.float32),
        "phi": tf.io.FixedLenFeature([], tf.float32),

        "category_id": tf.io.FixedLenFeature([], tf.int64),
        "id": tf.io.FixedLenFeature([], tf.int64),
        "image_id": tf.io.FixedLenFeature([], tf.int64),
        "is_crowd": tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, feature_description)
    example["image"] = tf.io.decode_png(example["image"], channels=3)
    return example

In [14]:
dataset = dict()
for key in sorted(images, key=lambda x: x['id']):
    dataset[key['id']] = dict()
    file_name = key['file_name']
    dataset[key['id']]['file_name'] = file_name

In [15]:
for annot in annotations:
    if 'image_id' not in dataset[annot['image_id']]:
        dataset[annot['image_id']]['image_id'] = annot['image_id']

        dataset[annot['image_id']]['area'] = [annot['area']]

        dataset[annot['image_id']]['row'] = [annot['bbox'][0]]
        dataset[annot['image_id']]['col'] = [annot['bbox'][1]]
        dataset[annot['image_id']]['width'] = [annot['bbox'][2]]
        dataset[annot['image_id']]['height'] = [annot['bbox'][3]]
        dataset[annot['image_id']]['phi'] = [annot['bbox'][4]]

        dataset[annot['image_id']]['category_id'] = [annot['category_id']]
        dataset[annot['image_id']]['id'] = [annot['id']]
        dataset[annot['image_id']]['is_crowd'] = [annot['is_crowd']]      

    else:
        dataset[annot['image_id']]['area'].append(annot['area'])

        dataset[annot['image_id']]['row'].append(annot['bbox'][0])
        dataset[annot['image_id']]['col'].append(annot['bbox'][1])
        dataset[annot['image_id']]['width'].append(annot['bbox'][2])
        dataset[annot['image_id']]['height'].append(annot['bbox'][3])
        dataset[annot['image_id']]['phi'].append(annot['bbox'][4])

        dataset[annot['image_id']]['category_id'].append(annot['category_id'])
        dataset[annot['image_id']]['id'].append(annot['id'])
        dataset[annot['image_id']]['is_crowd'].append(annot['is_crowd'])


In [16]:
if not os.path.exists(tfrecords_dir):
    os.makedirs(tfrecords_dir)  # creating TFRecords output folder
for tfrec_num in range(num_tfrecords):
    #samples = dataset[(tfrec_num * num_samples) : ((tfrec_num + 1) * num_samples)]
    samples = list(dataset.values())

    with tf.io.TFRecordWriter(
        tfrecords_dir + "/file_%.2i-%i.tfrec" % (tfrec_num, len(samples))
    ) as writer:
        for sample in samples:
            image_path = os.path.join(images_dir, sample['file_name'])
            image = tf.io.decode_png(tf.io.read_file(image_path))
            example = create_example(image, image_path, sample)
            writer.write(example.SerializeToString())

In [17]:
tfrecord_filename = tfrecords_dir + "/file_%.2i-%i.tfrec" % (tfrec_num, len(samples))
raw_data = tf.data.TFRecordDataset(tfrecord_filename)

for raw_record in raw_data.take(1):
  example = tf.train.Example()
  example.ParseFromString(raw_record.numpy())
  print(example)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

